In [70]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [53]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LogisticRegression
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, KFold
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer, SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("BreastCancer.csv", index_col= 0 )
df

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,Benign
63375,9,1,2,6,4,10,7,7,2,Malignant
76389,10,4,7,2,2,8,6,1,1,Malignant
95719,6,10,10,10,8,10,7,10,7,Malignant
128059,1,1,1,1,2,5,5,1,1,Benign
...,...,...,...,...,...,...,...,...,...,...
1369821,10,10,10,10,5,10,10,10,7,Malignant
1371026,5,10,10,10,4,10,5,6,3,Malignant
1371920,5,1,1,1,2,1,3,2,1,Benign


In [9]:
x, y = df.drop('Class', axis = 'columns'), df["Class"]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    random_state = 24,
                                                    test_size = 0.3,
                                                    stratify = y)


# Linear Kernel

In [43]:
svm = SVC(kernel = "linear",
         probability = True,
         random_state = 24)

svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9714285714285714


In [45]:
y_pred_prob = svm.predict_proba(x_test)[:, 1]

In [47]:
roc_auc_score(y_test, y_pred_prob)

0.9960748792270531

In [61]:

# 1. Imputation
# 2. OHE
# 3. scaleing
# 4. Model fitting

# ______________________________________________________________________________________
# Pipeline


lr = LogisticRegression(random_state = 24)

svm = SVC(kernel = "linear",
         random_state = 24,
         probability = True)

# n_components = number of most significant principle components to consider

lda = LinearDiscriminantAnalysis()

scl = StandardScaler().set_output(transform = "pandas")

pipe_pca = Pipeline([("SCL", scl), ("SVM", svm)])


# _____________________________________________________________________________________
# GCV

params = {'SVM__C' : np.linspace(0.0000001, 5, 20)}

kfolds = StratifiedKFold(n_splits = 5, 
                        random_state = 24, 
                        shuffle = True)

gcv = GridSearchCV(pipe_pca,
                  param_grid = params,
                  scoring = "roc_auc",
                  cv = kfolds,
                  verbose = 3)

gcv.fit(x, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ......................SVM__C=1e-07;, score=0.998 total time=   0.0s
[CV 2/5] END ......................SVM__C=1e-07;, score=0.995 total time=   0.0s
[CV 3/5] END ......................SVM__C=1e-07;, score=0.993 total time=   0.0s
[CV 4/5] END ......................SVM__C=1e-07;, score=0.988 total time=   0.0s
[CV 5/5] END ......................SVM__C=1e-07;, score=0.993 total time=   0.0s
[CV 1/5] END .........SVM__C=0.2631579894736842;, score=0.996 total time=   0.0s
[CV 2/5] END .........SVM__C=0.2631579894736842;, score=0.995 total time=   0.0s
[CV 3/5] END .........SVM__C=0.2631579894736842;, score=0.995 total time=   0.0s
[CV 4/5] END .........SVM__C=0.2631579894736842;, score=0.990 total time=   0.0s
[CV 5/5] END .........SVM__C=0.2631579894736842;, score=0.996 total time=   0.0s
[CV 1/5] END .........SVM__C=0.5263158789473683;, score=0.995 total time=   0.0s
[CV 2/5] END .........SVM__C=0.5263158789473683

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-07, 2.63157989e-01, 5.26315879e-01, 7.89473768e-01,
       1.05263166e+00, 1.31578955e+00, 1.57894744e+00, 1.84210533e+00,
       2.10526322e+00, 2.36842111e+00, 2.63157899e+00, 2.89473688e+00,
       3.15789477e+00, 3.42105266e+00, 3.68421055e+00, 3.94736844e+00,
       4.21052633e+00, 4.47368422e+00, 4.73684211e+00, 5.00000000e+00])},
             scoring='roc_auc', verbose=3)

In [67]:
y_pred = gcv.predict(x_test)
y_pred_prob = gcv.predict_proba(x_test)[:, 1]

print(f"Accuracy Score : {accuracy_score(y_test, y_pred)}")
print(f"ROC_AUC Score : {roc_auc_score(y_test, y_pred_prob)}")
print(f"Log loss Score : {log_loss(y_test, y_pred_prob)}")

Accuracy Score : 0.9714285714285714
ROC_AUC Score : 0.9961755233494365
Log loss Score : 0.08105352798541916


# Polynomial Kernel


In [73]:

# kernel "poly", default degree = 3
svm = SVC(kernel = "poly",
         probability = True,
         random_state = 24)

svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9714285714285714


In [75]:
y_pred_prob = svm.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, y_pred_prob)

0.9974838969404187

In [77]:
# ______________________________________________________________________________________
# Pipeline

scl = StandardScaler().set_output(transform = "pandas")

svm = SVC(kernel = "poly",
         random_state = 24,
         probability = True)

pipe_pca = Pipeline([("SCL", scl), ("SVM", svm)])


# _____________________________________________________________________________________
# GCV

params = {'SVM__C' : np.linspace(0.0000001, 5, 20),
          'SVM__degree' : [2, 3, 4]}

kfolds = StratifiedKFold(n_splits = 5, 
                        random_state = 24, 
                        shuffle = True)

gcv = GridSearchCV(pipe_pca,
                  param_grid = params,
                  scoring = "roc_auc",
                  cv = kfolds,
                  verbose = 3)

gcv.fit(x, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END .......SVM__C=1e-07, SVM__degree=2;, score=0.850 total time=   0.0s
[CV 2/5] END .......SVM__C=1e-07, SVM__degree=2;, score=0.719 total time=   0.0s
[CV 3/5] END .......SVM__C=1e-07, SVM__degree=2;, score=0.840 total time=   0.0s
[CV 4/5] END .......SVM__C=1e-07, SVM__degree=2;, score=0.824 total time=   0.0s
[CV 5/5] END .......SVM__C=1e-07, SVM__degree=2;, score=0.727 total time=   0.0s
[CV 1/5] END .......SVM__C=1e-07, SVM__degree=3;, score=0.998 total time=   0.0s
[CV 2/5] END .......SVM__C=1e-07, SVM__degree=3;, score=0.995 total time=   0.0s
[CV 3/5] END .......SVM__C=1e-07, SVM__degree=3;, score=0.993 total time=   0.0s
[CV 4/5] END .......SVM__C=1e-07, SVM__degree=3;, score=0.989 total time=   0.0s
[CV 5/5] END .......SVM__C=1e-07, SVM__degree=3;, score=0.992 total time=   0.0s
[CV 1/5] END .......SVM__C=1e-07, SVM__degree=4;, score=0.865 total time=   0.0s
[CV 2/5] END .......SVM__C=1e-07, SVM__degree=4

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(kernel='poly', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-07, 2.63157989e-01, 5.26315879e-01, 7.89473768e-01,
       1.05263166e+00, 1.31578955e+00, 1.57894744e+00, 1.84210533e+00,
       2.10526322e+00, 2.36842111e+00, 2.63157899e+00, 2.89473688e+00,
       3.15789477e+00, 3.42105266e+00, 3.68421055e+00, 3.94736844e+00,
       4.21052633e+00, 4.47368422e+00, 4.73684211e+00, 5.00000000e+00]),
                         'SVM__degree': [2, 3, 4]},
             scoring='roc_auc', verbose=3)

In [81]:
y_pred = gcv.predict(x_test)
y_pred_prob = gcv.predict_proba(x_test)[:, 1]

print(f"Best Score : {gcv.best_score_}")
print(f"Best Params : {gcv.best_params_}")

print(f"Accuracy Score : {accuracy_score(y_test, y_pred)}")
print(f"ROC_AUC Score : {roc_auc_score(y_test, y_pred_prob)}")
print(f"Log loss Score : {log_loss(y_test, y_pred_prob)}")

Best Score : 0.9957167518225637
Best Params : {'SVM__C': 3.4210526631578944, 'SVM__degree': 3}
Accuracy Score : 0.9952380952380953
ROC_AUC Score : 0.9996980676328503
Log loss Score : 0.050676664998974726


# Radial Kernel

In [84]:

svm = SVC(kernel = "rbf",
         probability = True,
         random_state = 24)

svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9761904761904762


In [86]:
y_pred_prob = svm.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, y_pred_prob)

0.9911433172302737

In [88]:
# ______________________________________________________________________________________
# Pipeline

scl = StandardScaler().set_output(transform = "pandas")

svm = SVC(kernel = "rbf",
         random_state = 24,
         probability = True)

pipe_pca = Pipeline([("SCL", scl), ("SVM", svm)])


# _____________________________________________________________________________________
# GCV

params = {'SVM__C' : np.linspace(0.0000001, 5, 20),
          'SVM__gamma' : np.linspace(0.0000001, 5, 20)}

kfolds = StratifiedKFold(n_splits = 5, 
                        random_state = 24, 
                        shuffle = True)

gcv = GridSearchCV(pipe_pca,
                  param_grid = params,
                  scoring = "roc_auc",
                  cv = kfolds,
                  verbose = 3)

gcv.fit(x, y)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
[CV 1/5] END ....SVM__C=1e-07, SVM__gamma=1e-07;, score=0.998 total time=   0.0s
[CV 2/5] END ....SVM__C=1e-07, SVM__gamma=1e-07;, score=0.995 total time=   0.0s
[CV 3/5] END ....SVM__C=1e-07, SVM__gamma=1e-07;, score=0.993 total time=   0.0s
[CV 4/5] END ....SVM__C=1e-07, SVM__gamma=1e-07;, score=0.988 total time=   0.0s
[CV 5/5] END ....SVM__C=1e-07, SVM__gamma=1e-07;, score=0.993 total time=   0.0s
[CV 1/5] END SVM__C=1e-07, SVM__gamma=0.2631579894736842;, score=0.993 total time=   0.0s
[CV 2/5] END SVM__C=1e-07, SVM__gamma=0.2631579894736842;, score=0.991 total time=   0.0s
[CV 3/5] END SVM__C=1e-07, SVM__gamma=0.2631579894736842;, score=0.983 total time=   0.0s
[CV 4/5] END SVM__C=1e-07, SVM__gamma=0.2631579894736842;, score=0.975 total time=   0.0s
[CV 5/5] END SVM__C=1e-07, SVM__gamma=0.2631579894736842;, score=0.990 total time=   0.0s
[CV 1/5] END SVM__C=1e-07, SVM__gamma=0.5263158789473683;, score=0.991 total time

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-07, 2.63157989e-01, 5.26315879e-01, 7.89473768e-01,
       1.05263166e+00, 1.31578955e+00, 1.57894744e+00, 1.84210533e+00,
       2.10526322e+00, 2.36842111e+00,...
                         'SVM__gamma': array([1.00000000e-07, 2.63157989e-01, 5.26315879e-01, 7.89473768e-01,
       1.05263166e+00, 1.31578955e+00, 1.57894744e+00, 1.84210533e+00,
       2.10526322e+00, 2.36842111e+00, 2.63157899e+00, 2.89473688e+00,
       3.15789477e+00, 3.42105266e+00, 3.68421055e+00, 3.94736844e+00,
       4.21052633e+00, 4.47368422e+00, 4.73684211e+00, 5.00000000e+00])},
             scoring='roc_auc', verbose=3)

In [92]:
y_pred = gcv.predict(x_test)
y_pred_prob = gcv.predict_proba(x_test)[:, 1]

print(f"Best Score : {gcv.best_score_}")
print(f"Best Params : {gcv.best_params_}\n")

print(f"Accuracy Score : {accuracy_score(y_test, y_pred)}")
print(f"ROC_AUC Score : {roc_auc_score(y_test, y_pred_prob)}")
print(f"Log loss Score : {log_loss(y_test, y_pred_prob)}")

Best Score : 0.9933043839848409
Best Params : {'SVM__C': 1e-07, 'SVM__gamma': 1e-07}

Accuracy Score : 0.6571428571428571
ROC_AUC Score : 0.9939613526570048
Log loss Score : 0.6429564828770069


# For HR Dataset

In [101]:
df = pd.read_csv("HR_comma_sep.csv")
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [103]:
x, y = df.drop("left", axis = 'columns'), df["left"]

In [105]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    random_state = 24,
                                                    test_size = 0.3,
                                                    stratify = y)

In [111]:
# ______________________________________________________________________________________
# Pipeline

# one hot encoder
ohe = OneHotEncoder(
    handle_unknown="ignore", 
    sparse_output=False, 
    drop='first'
).set_output(transform='pandas')

trans_ohe = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')


# scaler
scl_std = StandardScaler().set_output(transform = "pandas")
scl_mm = MinMaxScaler().set_output(transform = "pandas")


# model
svm = SVC(kernel = "linear",
         random_state = 24,
         probability = True)

pipe_pca = Pipeline([("OHE", trans_ohe), ("SCL", scl), ("SVM", svm)])


# _____________________________________________________________________________________
# GCV

params = {
    'SVM__C' : np.linspace(0.001, 5, 3),
    # 'SVM__gamma' : np.linspace(0.0000001, 5, 20),
}

kfolds = StratifiedKFold(n_splits = 5, 
                        random_state = 24, 
                        shuffle = True)

gcv = GridSearchCV(pipe_pca,
                  param_grid = params,
                  scoring = "roc_auc",
                  cv = kfolds,
                  verbose = 3)

gcv.fit(x, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ......................SVM__C=0.001;, score=0.804 total time=  25.9s
[CV 2/5] END ......................SVM__C=0.001;, score=0.792 total time=  25.1s
[CV 3/5] END ......................SVM__C=0.001;, score=0.800 total time=  25.7s
[CV 4/5] END ......................SVM__C=0.001;, score=0.796 total time=  25.3s
[CV 5/5] END ......................SVM__C=0.001;, score=0.812 total time=  25.6s
[CV 1/5] END .........SVM__C=2.5004999999999997;, score=0.808 total time=  46.9s
[CV 2/5] END .........SVM__C=2.5004999999999997;, score=0.801 total time=  46.3s
[CV 3/5] END .........SVM__C=2.5004999999999997;, score=0.809 total time=  49.0s
[CV 4/5] END .........SVM__C=2.5004999999999997;, score=0.802 total time=  46.7s
[CV 5/5] END .........SVM__C=2.5004999999999997;, score=0.818 total time=  47.8s
[CV 1/5] END ........................SVM__C=5.0;, score=0.808 total time= 1.0min
[CV 2/5] END ........................SVM__C=5.0;,

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002314194F860>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002313C0C4D10>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', StandardScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.0000e-03, 2.5005e+00, 5.0000e+00])},
             scoring='roc_auc', verbose=3)

In [113]:
y_pred = gcv.predict(x_test)
y_pred_prob = gcv.predict_proba(x_test)[:, 1]

print(f"Best Score : {gcv.best_score_}")
print(f"Best Params : {gcv.best_params_}\n")

print(f"Accuracy Score : {accuracy_score(y_test, y_pred)}")
print(f"ROC_AUC Score : {roc_auc_score(y_test, y_pred_prob)}")
print(f"Log loss Score : {log_loss(y_test, y_pred_prob)}")

Best Score : 0.8074981389904077
Best Params : {'SVM__C': 5.0}

Accuracy Score : 0.7755056679262058
ROC_AUC Score : 0.8024676821939315
Log loss Score : 0.4407688372073667
